In [1]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.8 MB/s eta 0:00:00


In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, pipeline
import torch

# === Setup ===
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device set to use {device}")

embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", device=device)
generation_model = "MBZUAI/LaMini-Flan-T5-783M"
tokenizer = AutoTokenizer.from_pretrained(generation_model)
qa_pipeline = pipeline("text2text-generation", model=generation_model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

# === Load and process case law ===
def load_case_law(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        case_blocks = content.strip().split('\n\n')  # Each case is a block
    return case_blocks

case_law_data = load_case_law("citations.txt")
case_embeddings = embedding_model.encode(case_law_data, convert_to_tensor=False)
case_embeddings = np.array(case_embeddings)

# === Build FAISS index ===
dimension = case_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(case_embeddings)

# === Define function to retrieve context ===
def get_context(question, top_k=3):
    question_embedding = embedding_model.encode([question])
    distances, indices = faiss_index.search(np.array(question_embedding), top_k)
    top_contexts = [case_law_data[idx] for idx in indices[0]]

    context = "\n\n".join([f"Case {i+1}:\n{ctx}" for i, ctx in enumerate(top_contexts)])
    return context, top_contexts

# === Define function to generate legal explanation ===
def generate_legal_ruling(context, question):
    prompt = f"""
You are an expert in Islamic Family Law in Pakistan.

Below are case law excerpts related to the topic "{question}":

{context}

Using ONLY the information above, provide a legal explanation or ruling for: "{question}".

Keep your response factual, concise, and legally grounded.
"""
    result = qa_pipeline(prompt.strip(), max_length=512)[0]['generated_text']
    return result.strip()


Device set to use cpu


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu



Top 3 Relevant Citations for 'Inheritance ruling':

Case 1:
Scenario: Inheritance Rights of Orphaned Grandchildren
Law Reference: Islamic Inheritance Law (Wasiyyat)
Text: Orphaned grandchildren can inherit through wasiyyat or family arrangement even though Islamic law bypasses them.
Case Law: Sajida Bibi v. Muhammad Akbar (2020 SCMR 980)
Summary: The court allowed property allocation through wasiyyat despite traditional Islamic inheritance limitations.

Case 2:
Scenario: Step-Children’s Inheritance Rights
Law Reference: Islamic Law of Inheritance
Text: Stepchildren are not entitled to inheritance unless adopted or through bequest.
Case Law: Ijaz v. Mst. Salma (2015 YLR 981)
Summary: The court held that stepchildren had no automatic inheritance rights without a registered will or adoption.

Case 3:
Scenario: Inheritance Rights of Orphaned Grandchildren
Law Reference: Section 4, Muslim Family Laws Ordinance, 1961
Text: Grandchildren whose parents die before the grandfather/mother can in

In [7]:

# === Run the system ===
if __name__ == "__main__":
    keyword = "Nikah"
    context, citations = get_context(keyword)

    print(f"\nTop 3 Relevant Citations for '{keyword}':\n")
    for i, citation in enumerate(citations, 1):
        print(f"Case {i}:\n{citation}\n")

    ruling = generate_legal_ruling(context, keyword)
    print("\nGenerated Legal Ruling:\n")
    print(ruling)



Top 3 Relevant Citations for 'Khula':

Case 1:
Scenario: Khula (Divorce by Wife Through Court)
Law Reference: Dissolution of Muslim Marriages Act, 1939, Section 2(viii)
Text: A woman is entitled to a judicial divorce if she proves that living with the husband causes extreme hardship or incompatibility.
Case Law: Khurshid Bibi v. Muhammad Amin (PLD 1967 SC 97)
Summary: The court granted khula on the basis that the wife had developed hatred and could not continue married life with the husband.

Case 2:
Scenario: Woman Seeking Khula Without Mehr Return
Law Reference: Dissolution of Muslim Marriages Act, 1939
Text: A woman seeking khula may not be required to return mehr if harm is proven.
Case Law: Shamim Akhtar v. Bashir Ahmad (2010 CLC 1483)
Summary: The court granted khula without returning mehr due to proven cruelty by the husband.

Case 3:
Scenario: Khula Granted Without Husband’s Consent
Law Reference: Dissolution of Muslim Marriages Act, 1939
Text: A woman can be granted khula wit